In [1]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
import numpy as np
import pandas as pd
import ntpath
import glob
import deepconfig

#from keras.utils.visualize_util import plot
#from IPython.display import Image, display

from CMS_SURF_2016.utils.preprocessing import *
from CMS_SURF_2016.utils.callbacks import OverfitStopping, SmartCheckpoint
from CMS_SURF_2016.utils.batch import batchAssertArchived, batchExecuteAndTestTrials
from CMS_SURF_2016.utils.archiving import *
from CMS_SURF_2016.layers.lorentz import Lorentz, _lorentz
from CMS_SURF_2016.layers.slice import Slice

from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, merge, Input, Flatten, Lambda, LSTM, Masking
from keras.engine.topology import Layer
from keras.callbacks import EarlyStopping
from keras.utils.visualize_util import plot
from keras.layers.advanced_activations import LeakyReLU


dc = deepconfig.deepconfig(gpu='gpu0', backend='theano')




Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)

Couldn't import dot_parser, loading of dot files will not be possible.
using gpu0
using theano


In [2]:
#The observables taken from the table
observ_types = ['Entry','E/c', 'Px', 'Py', 'Pz', 'PT_ET','Eta', 'Phi', 'Charge', 'X', 'Y', 'Z',\
                     'Dxy', 'Ehad', 'Eem', 'MuIso', 'EleIso', 'ChHadIso','NeuHadIso','GammaIso']
vecsize = len(observ_types)
epochs = 60
batch_size = 100

label_dir_pairs = \
            [   ("ttbar", "/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/"),
                ("wjet", "/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/"),
                ("qcd", "/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/")
            ]


In [3]:
def genModel(name,out_dim, depth, lstm_activation="relu", lstm_dropout = 0.0, dropout=0.0):
    inputs = []
    mergelist = []
    for i, profile in enumerate(object_profiles):
        inp = a = Input(shape=(profile.max_size, vecsize), name="input_"+str(i))
        inputs.append(inp)
        mergelist.append(a)
    a = merge(mergelist,mode='concat',concat_axis=1, name="merge")
    for i in range(depth):
        a = Masking(mask_value=0.0)(a)
        a = LSTM(vecsize,
                 input_shape=(None,vecsize),
                 dropout_W=lstm_dropout,
                 dropout_U=lstm_dropout,
                 activation=lstm_activation,
                 name = "lstm_" +str(i))(a)
        if(dropout > 0.0):
            a =  Dropout(dropout, name="dropout_"+str(i))(a)
    dense_out = Dense(out_dim, activation='sigmoid', name='main_output')(a)
    model = Model(input=inputs, output=dense_out, name=name)
    return model

In [4]:
archive_dir = "/data/shared/Delphes/keras_archive/"
patience = 10
earlyStopping = EarlyStopping(verbose=1, patience=patience)
for sort_on in ["PT_ET", "Phi", "Eta"]:
    object_profiles = [ObjectProfile("Electron",-1),
                        ObjectProfile("MuonTight", -1),
                        ObjectProfile("Photon", -1),
                        ObjectProfile("MissingET", 1),
                        ObjectProfile("EFlowPhoton",120, query="PT_ET > 1.5", sort_columns=[sort_on], sort_ascending=False), 
                        ObjectProfile("EFlowNeutralHadron",250, query="PT_ET > 1.5", sort_columns=[sort_on], sort_ascending=False), 
                        ObjectProfile("EFlowTrack",200, query="PT_ET > 1.5", sort_columns=[sort_on], sort_ascending=False)] 
    
    resolveProfileMaxes(object_profiles, label_dir_pairs)

    dps, l = getGensDefaultFormat(archive_dir, (75000,20000,20000), 135000, \
                         object_profiles,label_dir_pairs,observ_types,megabytes=500)

    dependencies = batchAssertArchived(dps)
    train, num_train = l[0]
    val,   num_val   = l[1]
    test,  num_test  = l[2]
    max_q_size = l[3]
    print("MAXQ: ",max_q_size)

    labels = [x[0] for x in label_dir_pairs]
    for name in ['LSTM']:
        for depth in [1]:
                for activation in ['tanh']:
                    for lstm_dropout in [0.0]:
                        for dropout in [0.0]:
                            activation_name = activation if isinstance(activation, str) \
                                                else activation.__name__

                            model = genModel(name, len(label_dir_pairs), depth, activation, lstm_dropout, dropout)

                            print(model.summary())

                            trial = KerasTrial(archive_dir, name=name, model=model)

                            trial.setTrain(train_procedure=train,
                                           samples_per_epoch=num_train
                                          )
                            trial.setValidation(val_procedure=val,
                                               nb_val_samples=num_val)
                            trial.setCompilation(loss='binary_crossentropy',
                                      optimizer='rmsprop',
                                      metrics=['accuracy']
                                          )
                            
                            trial.setFit_Generator( 
                                            nb_epoch=epochs,
                                            callbacks=[earlyStopping],
                                            max_q_size = max_q_size)



                            print("EXECUTE: ", name,labels, depth, activation_name)
                            trial.execute(custom_objects={"Lorentz":Lorentz,"Slice": Slice},
                                         train_arg_decode_func=label_dir_pairs_args_decoder,
                                         val_arg_decode_func=label_dir_pairs_args_decoder)


                            trial.test(test_proc=test,
                                         test_samples=num_test,
                                         custom_objects={"Lorentz":Lorentz,"Slice": Slice},
                                        arg_decode_func = label_dir_pairs_args_decoder)



                            trial.to_record({"lables": labels,
                                             "depth": depth,
                                             "sort_on" : sort_on,
                                             "activation": activation_name,
                                             "dropout":dropout,
                                             "lstm_dropout":lstm_dropout,
                                             "query": "PT_ET > 1.5",
                                             "patience" : patience
                                            })
                                       

                
        

    

1.83072
Generating DataProcedure in range(0,75000):
   From 3 labels in range(0,273) for 3x273 = 819 Samples
   From 3 labels in range(273,546) for 3x273 = 819 Samples
   From 3 labels in range(546,819) for 3x273 = 819 Samples
   From 3 labels in range(819,1092) for 3x273 = 819 Samples
   From 3 labels in range(1092,1365) for 3x273 = 819 Samples
   From 3 labels in range(1365,1638) for 3x273 = 819 Samples
   From 3 labels in range(1638,1911) for 3x273 = 819 Samples
   From 3 labels in range(1911,2184) for 3x273 = 819 Samples
   From 3 labels in range(2184,2457) for 3x273 = 819 Samples
   From 3 labels in range(2457,2730) for 3x273 = 819 Samples
   From 3 labels in range(2730,3003) for 3x273 = 819 Samples
   From 3 labels in range(3003,3276) for 3x273 = 819 Samples
   From 3 labels in range(3276,3549) for 3x273 = 819 Samples
   From 3 labels in range(3549,3822) for 3x273 = 819 Samples
   From 3 labels in range(3822,4095) for 3x273 = 819 Samples
   From 3 labels in range(4095,4368) for 3

KeyboardInterrupt: 